# Caso de estudio 1:
## Determinista & al menos uno de los dos subconjuntos de funciones correctas o incorrectas es singleton.
### Algoritmos genéticos


Se define una clase ProblemaGenetico que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los explicados en la memoria:

 - genes: lista de genes usados en el genotipo de los estados.
 - longitud_individuos: longitud de los cromosomas
 - decodifica: función de obtiene el fenotipo a partir del genotipo.
 - fitness: función de valoración.
 - muta: función de mutación de un cromosoma 
 - cruza: función de cruce de un par de cromosomas

In [1]:
import random

### INSTANCIA DEL PROBLEMA Y PRUEBAS
Necesitamos una nueva clase para este tipo de problemas ya que es necesario pasarle más información al instanciarlos. A parte de la información que ya le pasábamos antes, es necesario conocer el número de tests.

In [2]:
class ProblemaGenéticoCaso1(ProblemaGenetico):
    def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness, nTests):
        self.genes = genes
        self.fun_dec = fun_dec
        self.fun_cruza = fun_cruza
        self.fun_muta = fun_muta
        self.fun_fitness = fun_fitness
        self.longitud_individuos = nTests
        self.nTests = nTests #número de tests que tengo

        """Constructor de la clase"""
        
    def decodifica(self, genotipo):
        fenotipo = self.fun_dec(self.nTests, genotipo)
        """Devuelve el fenotipo a partir del genotipo"""
        return fenotipo
    
    def fitness(self, cromosoma):
        valoracion = self.fun_fitness(self.nTests, cromosoma)
        """Función de valoración"""
        return valoracion
    
    def cruza(self, cromosoma1, cromosoma2):
        cruce = self.fun_cruza(cromosoma1, cromosoma2) 
        """Devuelve el cruce de un par de cromosomas"""
        return cruce  #devuelve los dos cruces 

    def muta(self, cromosoma, prob):
        mutacion = self.fun_muta( self.nTests, cromosoma, prob)
        """Devuelve el cromosoma mutado"""   
        return mutacion
    
    
    
    

NameError: name 'ProblemaGenetico' is not defined

   - Definimos una función poblacion_inicial(problema_genetico,tamaño), para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior ProblemaGenetico

usamos random.choice

   - Definimos una función de cruce que recibe una instancia de Problema_Genetico y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

cruza_padres(problema_genetico,padres)

   - Definimos la función de mutación que recibe una instancia de Problema_Genetico, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.
muta_individuos(problema_genetico, poblacion, prob)

   - Definimos la función auxiliar nueva_generacion(problema_genetico,poblacion,n_padres,n_directos,prob_mutar) que dada una población calcula la siguiente generación.

In [ ]:
def cruza_padres(problema_genetico,padres):
    l = []
    n=len(padres)
    for i in range (n//2):
        cruces = problema_genetico.cruza( padres[2 * i], padres[2 * i + 1] )
        l.append(cruces[0].copy())
        l.append(cruces[1].copy())
    return l

In [3]:
def muta_individuos(problema_genetico, poblacion, prob):
    # hay que llamar a  problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
    for x in poblacion:
        problema_genetico.muta(x, prob)
    return poblacion

In [4]:
#Definir la función nueva_generacion
def nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1) #devuelve cromosomas
    generacion = padres2 + cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

In [5]:
def algoritmo_genetico_Caso1(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion = poblacion_inicial_Caso1(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres #los que pongo directamente sin cruzar
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness) #opt es max o min por ej, es una fun llega como arg
    mejor=problema_genetico.decodifica(mejor_cr)


    return (mejor,problema_genetico.fitness(mejor_cr)) 



### REPRESENTACIÓN:
Cada solución candidata o cromosoma es un vector de bits en binario, cada gen tieneun único alelo (bit) que representa si aplicamos ese test o no.

### POBLACIÓN INICIAL:
Número N de soluciones candidatas generadas aleatoriamente, los genes deben tener valores 0/1 y no otros.
Es decir, generamos listas de T elementos, cuyos valores son 0/1.
Tenemos un vector 

In [6]:
def poblacion_inicial_Caso1(problema_genetico, tam):
    n = tam #número de cromosomas
    numTestsTotal = problema_genetico.longitud_individuos
    genes = problema_genetico.genes 
    poblacion = list()
    for i in range(n): # cada cromosoma es una iteración
        poblacion.append( [random.choice(genes) for j in range(numTestsTotal)] ) #un cromosoma
    return poblacion


DUDA: si he definido que es de 0/1 el valor

### SELECCIÓN:
Usamos el algoritmo de selección por torneo 


In [7]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness) 
        #opt es por ejemplo max si es mejor ser máximo o min si minimo
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

### MUTACIÓN: 
La función de mutación muta uno de los genes cambiando su valor. El nuevo valor del gen mutado será el contrario 0/1. Por ello, podemos tener un cromosoma con un test más o un test menos.

In [8]:
def mutacion_caso1( nTests, cromosoma, prob):  
    #n es el tamaño del cromosoma, seleccionamos la posición que cambiamos
    p = random.randint(0, nTests - 1)
    if prob > random.uniform(0, 1): #solo mutamos según la probabilidad de mutación
        if(cromosoma[p] == 0): 
            cromosoma[p] = 1
        else: cromosoma[p] = 0
       
    return cromosoma




### CRUCE:
Usamos el algoritmo de cruce por dos puntos. 

In [9]:
import math
def cruzar_caso1(cromosoma1, cromosoma2):
    """Cruza los cromosomas: vamos a emplear cruce por 2 puntos."""
    # Puede generar individuos con más o menos tests aplicados que los padres, por combinarlos, 
    # pero la suma de tests realizados por los padres será la suma de tests realizada por los hijos
    l = len(cromosoma1) # por cómo está definido el problema ambos tienen la misma longitud, el número de tests que hay
    corte1 = random.randint(0, l-1) 
    corte2 = random.randint(corte1, l-1)
    cruce1 = cromosoma1[0:corte1]+cromosoma2[corte1:corte2+1]+cromosoma1[corte2+1:l]
    cruce2 = cromosoma2[0:corte1]+cromosoma1[corte1:corte2+1]+cromosoma2[corte2+1:l]
  

    return [cruce1, cruce2]


### DECODIFICACION:

En la función de decodificación, hacemos lo mismo que en la función fitness pero además sacamos la solución que hemos obtenido. Es decir, al final tenemos una tupla con una lista de tests y el número de tests que aplico de la forma:

([ test1, ... , testk], nºTests)



In [10]:
def decodifica_caso1(nTests, cromosoma):
    testsRealizo = 0
    fenotipo = list()
    for test in range (0, nTests):
        tomado = cromosoma[test] #si es 0 no la realizamos

        if tomado == 1:
            fenotipo.append( cromosoma[test] )

        testsRealizo += cromosoma[test] #nos indica cuántos realizamos 
        return (tuple(fenotipo), testsRealizo)
    

In [11]:
genes = [0,1]
nTests = 5




### FITNESS

Vamos a aplicar los tests que aparecen en nuestro cromosoma. Sumaremos al fitness el número de estos tests. Una vez todos estos tests sean alicados, sumaremos 1 por cada IUT restante del conjunto con más de un elemento (correct/incorrect). Es decir, de las que aún no han sido eliminadas, sumaremos max(|correct|,|incorrect|). Como tratamos con el caso no determinista, esto será suficiente.


#### ALGORITMO FITNESS: 


Vamos a recorrer el cromosoma: cada posición i = 1 representa que tomamos ese test. Como buscamos el caso peor, no importa el orden por ser el caso 1 determinista. 

In [12]:
def fun_fitness_caso1(cromosoma, correct, incorrect):
    # Nuestra función de fitness suma 1 por cada test que tomamos en el cromosoma  
    N = int(math.sqrt(len(cromosoma)))
    fitness = 0
    for i in range(cromosoma.size()):
        fitness += cromosoma[i] #vale sumarlo porque es 0 si no se toma el test
    if correct.size() == 1:
        fitness += incorrect.size()
    else: 
        fitness += correct.size()
    return fitness
